# Data structures

## The mesh

We make a distinction between the data structures we use for loading a mesh from disk, and the
computational mesh (variables) that the program operates on. For MiniCombust, we do not prescribe
the former (in fact, we recommend leveraging existing, tested implementations like libmesh that are
compatible with gmsh).

For the latter, we allow variation to test the effect of, say SoA vs AoS layout. PETSC-FUN3D

In general, meshes can represent variables in terms of cells and faces, or in terms of nodes (vertices) and edges
(links).
It's possible to have staggered approaches where a variable of interest is defined at the cell centre,
but flux variables are at nodes.

For our problem, we use

We consider only the 3D case for MiniCombust.

Some common things are precalculated

Faces have areas
Cell have volumes


## Representation for calculations
[BASED ON dolfyn.90 READGEOMETRY]
* For every cell we store its face (ids) and nodes [WHAT DO WE NEED NODAL VALUES FOR?]
* For every face we store its adjacent cell (ids)


Note that loading meshes, partitioning them and pre-calculating these properties of cells and volumes
are all uninteresting from a scaling point of view: they're a once-off upfront cost. It's feasible that
this would even be pre-processing before a 'run' of the simulation.  However, we measure the total 
up-front cost, in case this is smething that needd to be optimised.

We don't even specify that these calculations needs to happen in parallel for MiniCombust.

The long-running bit is the simulation.



## Distributed meshes

We use 'domain decomposition'

### Partitioning
Partitioning libraries like Parmetis partition graphs defined in terms of nodes and edges. 
The graph we want to partition in this case is not the same as the nodes and edges that make up the 
computational mesh. It's a description of how different cells are adjacent to each other.

Here's an example:
[SHOW A 3D MESH WITH CELLS AND FACES AND ITS CORRESPONDING GRAPH]

### Representing ghost cells

## Are the cells for the FVM the same as the cells for the particles?

## Assembling a linear system from the mesh
We'll use the industry standard PETSc to represent matrices and vectors for linear solver.
Ideally we want zero-copy (DLPack)?

How to go from mesh variables to the assembled matrix.
Sparse matrix, dense vector.
Possible for this to be matrix-free Jacobian, but maybe we just use PETSc.



An unstructured mesh consists of polyhedral cells. We can define these by the edges (vertexes), cells and the
faces (adjacent between two cells). See fig:

[Fig 3.1 a from https://www.ctcms.nist.gov/fipy/download/fipy-1.2.3.pdf]


https://scicomp.stackexchange.com/questions/4733/what-are-some-good-data-types-for-unstructured-cell-centered-fvm-cfd-code

MiniCombust is face-based, i.e. Cell Centered FVM (CC-FVM)

We want a distributed mesh where we can find the neighbouring faces

??

In FVM, we are concerned with the average value of a variable of interest within a control volume (i.e. cell). Thiis
average value is stored at the cell centre. Fluxes for a face are approximated using the values from the two cells adjacent to the face. This is a low-order approximation, but is simple to store and results in systems with low-bandwidth matrices.

We use co-located rather than staggered grids for pressure and velocity/heat?

Mesh elements (cells, faces, edges) have Globally unique IDs (over distributed memories)
Each element may also have a local ID, that is not globally unqiue.
This allows renumbering locally for more efficient data structure access.
Cells are grouped into locally owned an ghost cells


The computational mesh consists of polyhedral cells, the faces of which have maximum 4 vertices

Face connectivity is also stored as CSR for easy traversal.

Vertex connectivity connectivity is stored as a distributed adjacency list:

CSR?
 CSR xadj  vs adjncy
For each node we know whether it is local or ghost
For ghost nodes, we know the process it belongs to

Since is store as CSR, we can always trivially look up neighbour nodes of a cell based on it's position in CSR
Describe

We partitional the cells by building a graph where nodes are the cell ID and edges define adjaccent cells


Nodes are totally a thing. We rely on a nodes/edges thing to partition mesh and also what does CSR mean if not
nodes and edges.

(we use process ID rather than rank, but you can think of it as the same fr MPI)

In contrast to NetworkX (edges, vertices), meshes are defined by their faces.
IS THIS TRUE?

The canonical app uses libmesh or something instead of implementing this from scratch?


Inspired by the PETSC-FUN3D papers (https://www.mcs.anl.gov/research/projects/petsc-fun3d/Papers/sc00.pdf) we
use AoS ("Interlacing"), CSR, Structural Blocking, Edge Reordering (Reverse Cuthill-McKee)
But actually want MPI-OpenMP (Maybe?)

## Partitioning the mesh over nodes
Uses Parmetis to partition the mesh for a number of ranks.
Use ParMetis 
Using a partitioner such as parmetis or metis, the nodes of the mesh are partitioned 
What does this mean for faces and cells?
Who owns a face variable?


## Sparse matrices
Each timestep results in a linear system Ax = b where A is sparse.
How do we represent these?
Are we matrix free?

## Timesteps, iterations and sweeps
Timestep: delta T as system evolves
Sweeps: we improve non-linear PDE accuracy by running several sweeps of the solution for each timestep, subsituting the
previous solution value at the same time step value
Iterations: number of iterations of linear solver


## Boundary type
Inlet, Outlet, wall, symmety planes?


```Fortran
////
 type :: CellData
     real    :: x(3)                      ! center coordinates
     real    :: vol                       ! volume
     integer :: ctid = 1                  ! fluid type id as set in fluid table
   end type

///
```


## Particles
```Fortran
   type ParticleProp
     real    :: x0(3) = 0.0                        ! starting coordinates
     real    :: v0(3) = 0.0                        ! starting velocity
     real    :: x1(3) = 0.0                        ! current coordinates
     real    :: v1(3) = 0.0                        ! current velocity
     real    :: a1(3) = 0.0                        ! current acceleration
     integer :: cell0 =  0                         ! starting cell the particle is in
     integer :: cell1 =  0                         ! current cell the particle is in
     integer :: face  =  0                         ! current face the particle is on
     real    :: dens0 = -1.0  ! <= undefined flag  ! current density
     real    :: diam0 =  1.0
     real    :: mass0 =  1.0
     logical :: wall  = .false.
   end type
```


https://www.cfd-online.com/Wiki/Discretization_of_the_diffusion_term


```
// Read number of cells
read ScaleFactor [detail, ignore]
allocate the Cell array (each has 3d coords, vol and fluid type id)


for each cell
    fill in coords and vol, fuel type
apply scale factor to all geom components and volume

read maxfaces [maximum facces of any cell]
allocate NFaces array (int number of faces for each cell)
allocate CFace array (list of faces)

```

### Questions
Does each processor need to kmnow more that its global-local mappings (i.e. nobody needs the total mapping?)

In the design, can we use a master/workers approach so that the partitioning etc. is done in the master node (maybe in parallel?) and then the workers are started with their bits, already renumbered?

